<a href="https://colab.research.google.com/github/alaa126458/Data-analysis/blob/main/pima_diabetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
#pima diabetic using Naive Bayes

In [106]:
import csv
import random
import math
import pandas as np

In [107]:
from typing_extensions import ClassVar
def load(filename):
  dataset = pd.read_csv(filename) # Use pandas to read the CSV file, which handles headers correctly
  dataset = dataset.values.tolist() # Convert to a list of lists
  return dataset

In [108]:
def splitDataset(dataset,splitRatio):
  trainSize=int(len(dataset)*splitRatio)
  trainSet=[]
  copy=list(dataset)
  while len(trainSet)<trainSize:
    index=random.randrange(len(copy))
    trainSet.append(copy.pop(index))
  return [trainSet,copy]

In [109]:
def saperateByClass(dataset):
  separeted={}
  for i in range(len(dataset)):
    vector=dataset[i]
    if(vector[-1] not in separeted):
      separeted[vector[-1]]=[]
    separeted[vector[-1]].append(vector)
  return separeted

In [110]:
def mean(numbers):
  return sum(numbers)/float(len(numbers))

In [111]:
def stdev(numbers):
  avg=mean(numbers)
  variance=sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
  return math.sqrt(variance)

In [112]:
def summarize(dataset):
  summaries=[(mean(attribute),stdev(attribute)) for attribute in zip(*dataset)]
  del summaries[-1]
  return summaries

In [113]:
def summarizeByClass(dataset):
  separeted=saperateByClass(dataset)
  summaries={}
  for classValue,instances in separeted.items():
    summaries[classValue]=summarize(instances)
  return summaries

In [114]:
def calculateProbability(x,mean,stdev):
  exponent=math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
  return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [115]:
def calculateClassProbabilities(summaries,inputVector):
  probabilities={}
  for classValue,classSummaries in summaries.items():
    probabilities[classValue]=1
    for i in range(len(classSummaries)):
      mean,stdev=classSummaries[i]
      x=inputVector[i]
      probabilities[classValue]*=calculateProbability(x,mean,stdev)
    return probabilities

In [116]:
def predict(summaries,inputVector):
  probabilities=calculateClassProbabilities(summaries,inputVector)
  bestlabel,bestprob=None,-1
  for classValue,probability in probabilities.items():
    if bestlabel is None or probability>bestprob:
      bestprob=probability
      bestlabel=classValue
  return bestlabel

In [117]:
def getPredictions(summaries,testSet):
  predictions=[]
  for i in range(len(testSet)):
    result=predict(summaries,testSet[i])
    predictions.append(result)
  return predictions

In [118]:
def getAccuracy(testSet,predictions):
  correct=0
  for x in range(len(testSet)):
    if testSet[x][-1]==predictions[x]:
      correct+=1
  return (correct/float(len(testSet)))*100.0

In [122]:
def main():
  filename='diabetes.csv' # Added .csv extension to the filename
  splitRatio =0.67
  dataset=load(filename)
  trainingSet,testSet=splitDataset(dataset,splitRatio)
  print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset),len(trainingSet),len(testSet)))
  summaries=summarizeByClass(trainingSet) # Changed prsummaries to summaries
  predictions=getPredictions(summaries,testSet)
  accuracy=getAccuracy(testSet,predictions)
  print('Accuracy:{0}%'.format(accuracy))

main()

Split 768 rows into train=514 and test=254 rows
Accuracy:65.74803149606299%
